In [2]:
import os
from glob import glob
import shutil
import yaml


In [18]:
train_val_split_size = 0.2
dir_path = os.getcwd()
YOLO_settings_file_path = f'{dir_path}/dataset/custom_data.yaml'

In [14]:
# dataset/
# ├── images/
# │   ├── train/
# │   └── val/
# ├── labels/
# │   ├── train/
# │   └── val/
# └── custom_data.yaml

In [ ]:
# 1. read all images and labels pathes
# 2. Create a pull from that list
# 3. Devide images equally with classes label by 80/20 train/val

In [26]:
dataset_dir = f"{dir_path}/dataset"
dataset_image_dir = dataset_dir + "/images/"
dataset_labels_dir = dataset_dir + "/labels/"
image_format = ".jpg"
label_format = ".txt"

In [27]:
# create a folders for YOLO tree structure
for path in [dataset_image_dir, dataset_labels_dir]:
    for folder in ["/train", "/val"]:
        if not os.path.exists(path+folder):
            os.makedirs(path+folder)

In [19]:

labels_list = []
images_list = []

for image_path in glob(dataset_image_dir + "/*" +image_format):
    file_name = os.path.basename(image_path).replace(image_format, label_format)
    if os.path.exists(dataset_labels_dir + file_name):
        labels_list.append(dataset_labels_dir + file_name)
        images_list.append(image_path)

# if all images already in the right folders, but should be reset with different division
for folder in ["train", "val"]:
    print(dataset_image_dir + folder + "/*" +image_format)
    for image_path in glob(dataset_image_dir + folder + "/*" +image_format):
        file_name = os.path.basename(image_path).replace(image_format, label_format)
        label_full_path = dataset_labels_dir + folder + "/" + file_name
        if os.path.exists(label_full_path):
            labels_list.append(label_full_path)
            images_list.append(image_path)


print("amount images",len(images_list))
print("amount labels",len(labels_list))

if len(images_list) == 0 or len(images_list) != len(labels_list):
    raise ValueError("No images found ! labels and images doesnt match")


/home/denys/PycharmProjects/Classification-of-pigmented-skin-lesions/train/dataset/images/train/*.jpg
/home/denys/PycharmProjects/Classification-of-pigmented-skin-lesions/train/dataset/images/val/*.jpg
amount images 972
amount labels 972


In [20]:
# get images to right folders

# use a formula for get correct val images along all dataset
# iterator % full_length / full_length * 0.1
# will get a each element from the list when counter reached a part from list

counter_val = 0
counter_train = 0

for i in range(0, len(images_list)):

    image_file_name = os.path.basename(images_list[i])
    label_file_name = os.path.basename(labels_list[i])

    if i % int((len(images_list) / (len(images_list) * train_val_split_size))) == 0:
        # move file to definet folder
        shutil.move(images_list[i], dataset_image_dir + "/val/" + image_file_name)
        shutil.move(labels_list[i], dataset_labels_dir + "/val/" + label_file_name)
        counter_val+=1
    else:
        shutil.move(images_list[i], dataset_image_dir + "/train/" + image_file_name)
        shutil.move(labels_list[i], dataset_labels_dir + "/train/" + label_file_name)
        counter_train+=1

print(f"{counter_train} were added to /train folder" )
print(f"{counter_val} were added to /val folder" )

777 were added to /train folder
195 were added to /val folder


In [73]:

# train: /kaggle/working/ultralytics/mel/images/train/
# val:  /kaggle/working/ultralytics/mel/images/val/

# # number of classes
# nc: 7
# # class names
# names: ["AKIEC", "BCC", "BKL", "DF","MEL","NV","VASC"]


# Data to be written to the YAML file
data = {
    'train': dataset_image_dir + "train/",
    'val': dataset_image_dir + "val/",
    'nc': 7,
    'names': '["AKIEC", "BCC", "BKL", "DF","MEL","NV","VASC"]'
}

if os.path.exists(YOLO_settings_file_path):
    os.remove(YOLO_settings_file_path)
# Writing the data to a YAML file
with open(YOLO_settings_file_path, 'w+') as file:
    yaml.dump(data, file)

print(f"Data has been written to '{YOLO_settings_file_path}'")

Data has been written to '/home/denys/PycharmProjects/Classification-of-pigmented-skin-lesions/train/dataset/custom_data.yaml'
